In [1]:
# coding: utf-8
import re
import hashlib
import numpy as np
import glob
import nltk
import chardet

## Directorios de lectura y escritura

In [2]:
movies = glob.glob('C:/Users/hp/Documents/Movies/datasets/peli_*')
root_scripts = 'C:/Users/hp/Documents/Movies/clean_scripts/'
root_personajes = 'C:/Users/hp/Documents/Movies/personajes/'
root_dialogos = 'C:/Users/hp/Documents/Movies/dialogos/'

## Glosarios

In [3]:
annotations=['INT', 'EXT', 'END', 'FADE', 'THE', 'OF', 'CUT', 'SCENE', 'TITLE', 'SCENES', 'TIGHT', 'DISSOLVE', 
             'OK', 'PHOTO', 'SCREEN', 'INTERCUT', 'SHOT', 'CLOSE']
nouns = ['NN', 'NNP', 'NNS']
N = set(nouns)
A = set(annotations)

## Funciones: limpiar guión, guardar archivo

In [4]:
def save(file, prefix, sufix): 
    name = prefix + sufix
    newfile = open(name, 'w')
    for line in file:
        newfile.write(u'%s \n' % line)
    newfile.close()

In [5]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def clean(script):
    cleanmovie=[]
    for line in script:
        line = cleanhtml(line)
        line = re.sub(r'\n', '', line)
        cleanmovie= np.append(cleanmovie, line.strip())
    cleanmovie=cleanmovie.tolist()
    return (cleanmovie)

In [6]:
def find_all_capital(line):
    patt = re.findall(r'[A-Z]+', line); discriminant = re.findall(r'[a-z]+', line) + re.findall(r'^[0-9]', line)
    aux=''
    if(patt and not discriminant): 
        for string in patt:
            if len(string)>1 : aux = aux + ' ' +string.strip()
        return aux
    else: return False

def find_one_capital(line):
    patt = 0
    if line.count(' ')==1: patt = re.findall(r'[A-Z][a-z]+', line); discriminant = re.findall(r'(?<= )+[a-z]+\b', line)
        
    aux=''
    if(patt and not discriminant): 
        for string in patt:
            if len(string)>1 : aux = aux + ' ' +string.strip()
        return aux
    else: return False

In [7]:
def find_nouns(line):
    line=line.lower()
    tokens = nltk.word_tokenize(line)
    token_tags=nltk.pos_tag(tokens)
    tags = []
    for touple in token_tags: tags = tags + [touple[1]]
    TA = set(tags)
    if(TA.intersection(N)): return 1
    else: return 0

In [8]:
def all_cap_characters(script):
    lines = []
    for i,line in enumerate(script):
        character = find_all_capital(line)
        if(character):
            tokens = nltk.word_tokenize(character)
            T = set(tokens)
            flag_1 = T.intersection(A)
            flag_2 = character.count(' ') < 3
            flag_3 = find_nouns(character)
            if(not flag_1 and flag_2 and flag_3):  lines = lines+[str(i)+' '+character.strip()]
    return lines

def one_cap_characters(script):
    lines = []
    for i,line in enumerate(script):
        character = find_one_capital(line)
        if(character):
            tokens = nltk.word_tokenize(character)
            T = set(tokens)
            flag_1 = T.intersection(A)
            flag_2 = character.count(' ') < 3
            flag_3 = find_nouns(character)
            if(not flag_1 and flag_2 and flag_3):  lines = lines +[str(i)+' '+character.strip()] 
    return lines

In [9]:
def characters(script):
    all_cap = all_cap_characters(script)
    one_cap = one_cap_characters(script)
    if(len(all_cap) > len(one_cap)): return all_cap
    else: return one_cap

In [10]:
def dialogues(script, characters):
    lineas=[]
    for idx, line in enumerate(characters #el enumerate es nuevos
        flag = 1
        nline = int(re.findall(r'^[0-9]+', line)[0])
        nextnline = int(re.findall(r'^[0-9]+', characters[(idx+1)% len(characters)])[0])
        personaje = re.sub(r'[0-9]+', '', line)
        dialogue = ''
        len_max = len(script)
        while(flag):
            dialogue = dialogue + ' '+ script[nline] #hacer nline+1 para quitar el personaje
            if((nline+1 < len_max) & (nline+1 < nextnline)): flag = len(script[nline+1])
            else: flag = 0
            nline += 1
        lineas = np.append(lineas, dialogue)
    lineas = list(lineas)
    return lineas

In [12]:
def title(script):
    for line in script:
        title = re.findall(r'[A-Z]+', line)
        if len(title)!=0: break
    title = ' '.join(title)  
    return title

def prefijo(cadena, n=10):
    return hashlib.md5(cadena.encode()).hexdigest()[:n]

## Limpieza, extracción de personajes y extracción de diálogo

In [164]:
for movie in movies[-1:-0]:
    dic = chardet.detect(open(movie, 'rb').read())
    fmovie = open(movie,'r', encoding = dic['encoding']).readlines()
    
    cmovie = clean(fmovie)
    personajes = characters(cmovie)
    dialogos = dialogues(cmovie, personajes)
    
    titulo = title(cmovie)
    hashname = prefijo(titulo)
    
    save(cmovie,     root_scripts,    hashname+'_guion.txt')
    save(personajes, root_personajes, hashname+'_personajes.txt')
    save(dialogos,   root_dialogos,   hashname+'_dialogos.txt')

In [165]:
not_running =[movies[36], movies[61], movies[130],movies[136],movies[167],movies[172],movies[209],movies[224],
      movies[273],movies[279],movies[284],movies[298],movies[314],movies[374],movies[397],movies[410],
      movies[422],movies[424],movies[508],movies[527],movies[556],movies[566],movies[568],movies[624],
      movies[625],movies[629],movies[729],movies[756],movies[820],movies[827],movies[900],movies[906],
      movies[913],movies[958],movies[988],movies[1012],movies[1024],movies[1033], movies[1048], movies[1050],
      movies[1085]]
len(not_running)

41